<a href="https://colab.research.google.com/github/LazaroR-u/qosf_2024_1/blob/main/task1_qosf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK I QOSF

Task 1 less than k

Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case




```
def less_than_k (int:k, list[int]: list_n):
     “””
k : integer value that is the positive number to compare in list_n,
list_n : integer list that has positive numbers.
Return the numbers that are in list_n and are less than k
     “””
```



     # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc.

      # consider print your quantum circuit.


Example:



```
A = less_than_k (7,[4,9,11,14,1,13,6,15])
print(A)
```

```
>>>> “4,1,6”
```



### Solution:

We aim to construct a function that returns numbers from a list that are less than a certain number $k$. To achieve this, we will follow these steps:

* Build a function to convert a number to binary format.
* Define a function that compares two binary numbers of $n$ digits using a quantum circuit.
  - First, we will use a circuit to compare each digit in a binary number.
  - Second, we will concatenate the differences found between the digits of two binary numbers.
  - Third, the circuit should return one of the following 3 states: the first number is greater than the second, the second number is greater than the first, or both numbers are equal.
* Utilize this function to compare the fixed number $k$ and each number in the list. This function should return a list with numbers less than the fixed number.


In [1]:
%%capture
pip install qiskit

In [2]:
%%capture
pip install -U qiskit-aer

In [3]:
from qiskit_aer import Aer
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import warnings
warnings.filterwarnings("ignore", message="Conversion of an array with ndim > 0 to a scalar is deprecated.*")


In [4]:
# Function to convert an integer to binary format with zero padding.
# Value: the integer to be converted.
# Zeros: the number of zeros to add for padding the binary format.
def int_to_bin(value: int, zeros: int):
    return bin(value)[2:].zfill(zeros)

# Function to encode a single bit into a quantum circuit.
# Bit: the bit to encode, can be '0' or '1'.
def encode(bit):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

# Function to return a quantum circuit for subtracting two bits.
# This quantum circuit compares two bits and generates a state indicating their relationship.
# There are four possible outcomes:
# 1. If both bits are 0, the output is 000.
# 2. If both bits are 1, the output is 100.
# 3. If the first bit is 1 and the second is 0, the output is 111.
# 4. If the first bit is 0 and the second is 1, the output is 010.
# The second qubit indicates whether the two bits are equal (0) or different (1).
# The third qubit indicates whether the first bit is greater than the second (1) or not (0).
def bit_subtract():
    bits = QuantumRegister(2, "bits")
    out = QuantumRegister(1, "out")

    qc = QuantumCircuit(bits, out)
    qc.cx(bits[0], bits[1])  # Compare the two bits
    qc.mcx(bits, out)

    return qc


In [5]:
def compare_bitstring(bitstring_a, bitstring_b, exec=True):
    # Determine the number of bits in the bitstring
    bits = len(bitstring_b)

    # Define Quantum Registers
    qra = QuantumRegister(bits, "a")
    qrb = QuantumRegister(bits, "b")
    qrcarry = QuantumRegister(bits, "carry")
    qreq = QuantumRegister(1, "equal")

    # Define Classical Register for measurement
    cr = ClassicalRegister(2)

    # Create Quantum Circuit
    qc = QuantumCircuit(qra, qrb, qrcarry, qreq, cr)

    # Encoding input bitstrings into quantum states
    for i in range(bits):
        qc.append(encode(bitstring_a[::-1][i]), [qra[i]])
        qc.append(encode(bitstring_b[::-1][i]), [qrb[i]])

        # Subtracting bits and handling carries
        if i > 0:
            qc.append(bit_subtract(), [qrcarry[i-1], qrb[i], qrcarry[i]])
        qc.append(bit_subtract(), [qra[i], qrb[i], qrcarry[i]])

    # Inverting the second bitstring for comparison
    for i in range(bits):
        qc.x(qrb[i])

    # Performing a multi-controlled X gate to check if all bits are equal
    qc.mcx([*qrb], qreq)

    # Measure the carry and equality registers
    qc.measure(qrcarry[bits-1], cr[0])
    qc.measure(qreq, cr[1])

    # Tell Qiskit how to simulate our circuit
    backend = Aer.get_backend('qasm_simulator')

    # Do the simulation, returning the result
    new_circuit = transpile(qc, backend)
    result = backend.run(new_circuit, shots=1000).result()

    # Get the probability distribution
    counts = result.get_counts()

    # The results could be the following:
    # '01' if the first bitstring is greater than the second
    # '10' if the first bitstring is equal to the second
    # '00' if the first bitstring is less than the second
    return counts




In [6]:
def less_than_k(k, list_n):
    """
    Compares each value in a list with a reference fixed value k using the compare_bitstring function.
    k : integer value that is the positive number to compare in list_n,
    list_n : integer list that has positive numbers.
    Return a list with the numbers that are in list_n and are less than k
    """
    results = []
    output = []

    # Determine the number of bits needed for comparison
    max_value = max(max(list_n), k + 1)
    num_bits = int(np.ceil(np.log2(max_value)))

    # Compare each value with the reference value k
    for val in list_n:
        # Convert values to binary strings
        val_binary = int_to_bin(val, num_bits)
        ref_binary = int_to_bin(k, num_bits)
        # Compare binary strings using the compare_bitstring function
        results.append(compare_bitstring(ref_binary, val_binary))

    # Collect values less than the reference value
    for i, result in enumerate(results):
        if result.get('01') == 1000:
            output.append(list_n[i])

    return output


In [7]:
values = [1,2,3,3,4,5,6,7,8,9,10, 11, 12, 13 ,15]
k = 7
less_than_k(k,values)

[1, 2, 3, 3, 4, 5, 6]

**Conclusion**:

Through this exercise, we successfully implemented a quantum algorithm aimed at identifying integers within a list that are less than a fixed number $k$. Leveraging binary numbers allowed for a straightforward encoding of integers into quantum circuits.

A notable advantage of this approach is its purely quantum nature, eliminating the need to preselect numbers less than $k$. The quantum circuit autonomously conducts comparisons between pairs of numbers, streamlining the process.

This method showcased efficient comparison and evaluation of integer values, highlighting the adaptability of quantum computing in handling integer-based tasks. It's worth noting that due to limitations in the "qasm_simulator," we're constrained to comparing numbers with a maximum length of 9 bits, equating to the number 511.

However, as the number of bits increases, so does the computational complexity. Hence, exploring alternative approaches to alleviate this computational burden and bolster efficiency may be worthwhile.


References:

[1] Deutsch, David, and Richard Jozsa. "Rapid solution of problems by quantum computation." Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences 439.1907 (1992): 553-558.

[2] Bernstein, Ethan, and Umesh Vazirani. "Quantum complexity theory." SIAM Journal on computing 26.5 (1997): 1411-1473.

[3] Grover, Lov K. , "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (1996), arXiv:quant-ph/9605043

[4] Zicker Frank. "A Beginner-Friendly Quantum Algorithm", Medium. (2023)
https://pyqml.medium.com/a-beginner-friendly-quantum-algorithm-9d7b32e575b5

## extra

Comparing results with classical operation

In [14]:
import random

n = 10
maximo = 256
values = [random.randint(1, maximo) for _ in range(n)]
k = random.randint(1, maximo)
classical_solution = [val for val in values if val < k]
quantum_solution = less_than_k(k,values)

print(classical_solution)
print(quantum_solution)

[14, 9, 4, 20]
[14, 9, 4, 20]


In [10]:
# to compare 499 with 400 or 500 we need near of 30 qubits. This makes circuit computationally expensive.
less_than_k(499, [400,500])

[400]

About the quantum circuit

In [11]:
def compare_bitstring_circuit(bitstring_a, bitstring_b, exec=True):

    bits = len(bitstring_b)
    qra = QuantumRegister(bits, "a")
    qrb = QuantumRegister(bits, "b")
    qrcarry = QuantumRegister(bits, "carry")
    qreq = QuantumRegister(1, "equal")

    cr = ClassicalRegister(2)

    qc = QuantumCircuit(qra, qrb, qrcarry, qreq, cr)

    for i in range(bits):
        qc.append(encode(bitstring_a[::-1][i]), [qra[i]])
        qc.append(encode(bitstring_b[::-1][i]), [qrb[i]])

        if i > 0:
            qc.append(bit_subtract(), [ qrcarry[i-1], qrb[i], qrcarry[i] ])

        qc.append(bit_subtract(), [qra[i], qrb[i], qrcarry[i]])

    for i in range(bits):
        qc.x(qrb[i])

    qc.mcx([*qrb], qreq)

    qc.measure(qrcarry[bits-1], cr[0])
    qc.measure(qreq, cr[1])

    return qc

In [12]:
qc = compare_bitstring_circuit("1", "0")
qc.draw()

┌──────────────┐┌───────────────┐             
    a: ┤ circuit-1191 ├┤0              ├─────────────
       ├──────────────┤│               │┌───┐        
    b: ┤ circuit-1194 ├┤1 circuit-1197 ├┤ X ├──■─────
       └──────────────┘│               │└┬─┬┘  │     
carry: ────────────────┤2              ├─┤M├───┼─────
                       └───────────────┘ └╥┘ ┌─┴─┐┌─┐
equal: ───────────────────────────────────╫──┤ X ├┤M├
                                          ║  └───┘└╥┘
c18: 2/═══════════════════════════════════╩════════╩═
                                          0        1

In [13]:
qc = compare_bitstring_circuit("100", "001")
qc.draw()

┌──────────────┐┌───────────────┐                                  »
    a_0: ┤ circuit-1201 ├┤0              ├──────────────────────────────────»
         ├──────────────┤│               │                 ┌───────────────┐»
    a_1: ┤ circuit-1210 ├┤               ├─────────────────┤0              ├»
         ├──────────────┤│               │                 │               │»
    a_2: ┤ circuit-1222 ├┤               ├─────────────────┤               ├»
         ├──────────────┤│               │      ┌───┐      │               │»
    b_0: ┤ circuit-1204 ├┤1 circuit-1207 ├──────┤ X ├──────┤               ├»
         ├──────────────┤│               │┌─────┴───┴─────┐│               │»
    b_1: ┤ circuit-1213 ├┤               ├┤1              ├┤1 circuit-1219 ├»
         ├──────────────┤│               ││               ││               │»
    b_2: ┤ circuit-1225 ├┤               ├┤               ├┤               ├»
         └──────────────┘│               ││  circuit-1216 ││               │»
carry_0: ────────────────┤2              ├┤0              ├┤               ├»
                         └───────────────┘│               ││               │»
carry_1: ─────────────────────────────────┤2              ├┤2              ├»
                                          └───────────────┘└───────────────┘»
carry_2: ───────────────────────────────────────────────────────────────────»
                                                                            »
  equal: ───────────────────────────────────────────────────────────────────»
                                                                            »
  c19: 2/═══════════════════════════════════════════════════════════════════»
                                                                            »
«                                                        
«    a_0: ───────────────────────────────────────────────
«                                                        
«    a_1: ───────────────────────────────────────────────
«                          ┌───────────────┐             
«    a_2: ─────────────────┤0              ├─────────────
«                          │               │             
«    b_0: ─────────────────┤               ├───────■─────
«               ┌───┐      │               │       │     
«    b_1: ──────┤ X ├──────┤               ├───────■─────
«         ┌─────┴───┴─────┐│               │┌───┐  │     
«    b_2: ┤1              ├┤1 circuit-1231 ├┤ X ├──■─────
«         │               ││               │└───┘  │     
«carry_0: ┤               ├┤               ├───────┼─────
«         │  circuit-1228 ││               │       │     
«carry_1: ┤0              ├┤               ├───────┼─────
«         │               ││               │ ┌─┐   │     
«carry_2: ┤2              ├┤2              ├─┤M├───┼─────
«         └───────────────┘└───────────────┘ └╥┘ ┌─┴─┐┌─┐
«  equal: ────────────────────────────────────╫──┤ X ├┤M├
«                                             ║  └───┘└╥┘
«  c19: 2/════════════════════════════════════╩════════╩═
«                                             0        1

This compare bitstring circuit utilizes $3n + 1$ qubits, where $n$ represents the number of bits in the binary number being compared. For instance, when comparing the numbers 3 and 4, their corresponding binary representations are "011" and "100". In this case, $n = 3$, requiring $3 \times 3 + 1 = 10$ qubits in the quantum circuit.